In [3]:
from gurobipy import *
import pandas as pd

distance_matrix = pd.read_csv('/Users/jasper/Operation Research/Final_Project/distance_matrix.csv')

nodes = []
for i in range(len(distance_matrix)):
    nodes.append(str(i+1))
distance = {}
for i in range(distance_matrix.shape[0]):
    for j in range(distance_matrix.shape[1]):
        distance[(str(i+1),str(j+1))] = distance_matrix.iloc[i,j]
arcs, distance = multidict(distance)
# Create optimization model
m = Model('TSP')
# Create variables
x = {}
u = {}
for i,j in arcs:
    x[i,j] = m.addVar(obj=distance[i,j], vtype = 'B',
       name='x_%s%s' % (i, j))
N = len(nodes)
for i in nodes:
    if i != nodes[N-1]:
        u[i] = m.addVar(obj=0, name='u_%s' % i)
m.update()
# Constraint for sum of incoming links to j
for j in nodes:
     m.addConstr(quicksum(x[i,j]
               for i in nodes if i != j) == 1,
                    'incom_%s' % (j))
# Constraint for sum of outgoing links from i
for i in nodes:
     m.addConstr(quicksum(x[i,j]
               for j in nodes if i != j) == 1,
                    'outgo_%s' % (i))
# Subtour elimination constraints
for i,j in arcs:
    if i != nodes[N-1] and j != nodes[N-1]:
       m.addConstr(u[i] - u[j] + N*x[i,j] <= N-1,
                'subtour_%s_%s' % (i, j))
# Compute optimal solution
m.optimize()
# Print solution
route = []
if m.status == GRB.Status.OPTIMAL:
    print('objective: %f' % m.ObjVal)
    solution = m.getAttr('x', x)
    for i,j in arcs:
         if solution[i,j] > 0:
                print('%s -> %s: %g' % (i, j, solution[i,j]))
                route.append([i,j])
print(route)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4901 rows, 4969 columns and 23805 nonzeros
Model fingerprint: 0xf223858a
Variable types: 69 continuous, 4900 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [3e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 69 rows and 70 columns
Presolve time: 0.02s
Presolved: 4832 rows, 4899 columns, 23736 nonzeros
Variable types: 69 continuous, 4830 integer (4830 binary)

Root relaxation: objective 7.119086e+03, 231 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7119.08571    0  128          - 7119.08571      -     -    0s
     0     0 7737.00000    0  133    

In [4]:
route = pd.DataFrame(route)
route.to_csv('/Users/jasper/Operation Research/Final_Project/route_no_weight.csv', index=False, header=False)